# Cервис для чтения книг по подписке

Компания купила крупный сервис для чтения книг по подписке. Необходимо проанализировать базу данных, в которой содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

<b>Задания:</b>
* Посчитайте, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитайте количество обзоров и среднюю оценку;
* Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
* Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:

query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Cколько книг вышло после 1 января 2000 года

In [7]:
query = '''SELECT 
           COUNT (book_id) AS book_cnt
           FROM books 
           WHERE publication_date >= '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_cnt
0,821


По разным оценкам, в год издается около одного миллиона книг, примерно 0,6-0,9% из них новые, таким образом база данных сервиса содержит достаточно малое количество кних изданных за последние 20 лет

### Расчет для каждой книги количества обзоров и средней оценки

In [8]:
query = '''SELECT 
b.title,
COUNT (DISTINCT rv.review_id) AS review_cnt,
AVG(rt.rating) AS mean_rating

FROM books AS b
LEFT JOIN reviews AS rv ON b.book_id=rv.book_id
LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
GROUP BY b.title
ORDER BY review_cnt DESC
LIMIT 10;'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,review_cnt,mean_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,The Curious Incident of the Dog in the Night-Time,6,4.081081
3,Outlander (Outlander #1),6,4.125000
4,The Book Thief,6,4.264151
5,The Catcher in the Rye,6,3.825581
6,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
7,Eat Pray Love,6,3.395833
8,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
9,The Alchemist,6,3.789474


Обзоров у книг очень мало для объективной оценки

###  Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
query =  '''SELECT 
            p.publisher,
            COUNT (DISTINCT b.book_id) AS book_cnt

            FROM books AS b
            LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
            WHERE num_pages > 50
            GROUP BY p.publisher
            ORDER BY book_cnt DESC
            LIMIT 1;'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,book_cnt
0,Penguin Books,42


Опять же данные свидетельствуют о том, что сервис обладает небольшим банком книг

### Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;

In [10]:
query = '''WITH ar AS (SELECT b.book_id,
                       b.title,
                       a.author,
                       COUNT(rt.rating_id) AS rating_cnt,
                       AVG(rt.rating) AS avg_rating
                       FROM books AS b
                       LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
                       LEFT JOIN authors AS a ON b.author_id = a.author_id
                       GROUP BY a.author, b.book_id
                       HAVING COUNT(rt.rating_id) >=50)
            SELECT author,
            AVG(avg_rating)
            FROM ar
            GROUP BY author
            ORDER BY AVG(avg_rating) DESC
            LIMIT 1;'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Здесь согласимся, по многим оценкам авторитетных источников, книги Джоан Роулинг входят в списки лучших книг, наличие книг этого автора значительно повышает ценность сервиса

### Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
query = '''WITH us AS (SELECT username,
                      COUNT(rating_id) AS rating_cnt
                      FROM ratings
                      GROUP BY username
                      HAVING COUNT(rating_id) >48),
               rv AS (SELECT us.username,
                      COUNT(r.review_id) AS review_cnt
                      FROM us
                      LEFT JOIN reviews AS r ON us.username = r.username
                      GROUP BY us.username)
            SELECT AVG(review_cnt) AS avg_cnt_review
            FROM rv;'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_cnt_review
0,24.0


Опять же сервис обладает достаточно малой базой оценок и обзоров пользователей, это плохо сказывается на ценностой оценке сервиса

Мы ознакомились со всеми датафреймами в базе данных сервиса, выявили сколько книг, выпущеных после 1 января 2000 года, содержит база сервиса - 821 книгу, по разным оценкам, в год издается около одного миллиона книг, примерно 0,6-0,9% из них новые, таким образом база данных сервиса содержит достаточно малое количество кних изданных за последние 20 лет.

Мы рассчитали для каждой книги количество обзоров и среднюю оценку, оказалось, что самое большое количество обзоров у одной книги в базе данных сервиса всего 8, таким образом, обзоров у книг очень мало для объективной оценки.

В ходе исследования мы также рассмотрели у какого издательства содержится наибольшее количество книг в базе сервиса (за исключением брошюр), самое большое количество книг у издательства Penguin Books - 42 книги, опять же отмечаем, что объем изданий в базе сервиса, крайне ограниченный, соответственно мы вряд ли можем расчитывать на широкую аудиторию пользователей. 

Мы так же выяснили, у кого из авторов самая высокая средняя оценка книг (учитывая только книги с 50 и более оценками) в сервисе самые высокая средняя оценка у Джоан Роулинг, по многим оценкам авторитетных источников, книги Джоан Роулинг входят в списки лучших книг, наличие книг этого автора значительно повышает ценность сервиса.
Так же рассчитали среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора, сервис обладает достаточно малой базой оценок и обзоров пользователей, это плохо сказывается на ценностой оценке сервиса.
По итогам выполнения задач, невозможно сформулировать ценностное предложение для нового продукта, ряд факторов, таких как ограниченность базы данных книг, небольшое количество обзоров, негативно влияют на пользовательский интерес к данному продукту.